In [1]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt

In [2]:
def makeH_2_4(C, numem, spread, g, J, w, points):
    P = 0.01
    kappa = 10.
    tau = 5.8
        
    gamma = 1/tau

    if J == 1:
        w_list = np.linspace(-15, -15, points)
    if J == 5:
        w_list = np.linspace(-40, 40, points)    
    if J == 10:
        w_list = np.linspace(-40, 40, points)
    else:
        w_list = np.linspace(-200, 200, points)

    
    we_list = np.random.normal(w, scale = spread/2, size = C*numem)
    # w_list = np.linspace(-4*gcalcs[0], 4*gcalcs[0], points)
    a = []
    a.append(tensor(tensor(destroy(2), qeye(2), qeye(2)), tensor(qeye(2), qeye(2), qeye(2))))
    a.append(tensor(tensor(qeye(2), qeye(2), qeye(2)), tensor(destroy(2), qeye(2), qeye(2))))
    
    s = []
    s.append(tensor(tensor(qeye(2), destroy(2), qeye(2)), tensor(qeye(2), qeye(2), qeye(2))))
    s.append(tensor(tensor(qeye(2), qeye(2), destroy(2)), tensor(qeye(2), qeye(2), qeye(2))))
    s.append(tensor(tensor(qeye(2), qeye(2), qeye(2)), tensor(qeye(2), destroy(2), qeye(2))))
    s.append(tensor(tensor(qeye(2), qeye(2), qeye(2)), tensor(qeye(2), qeye(2), destroy(2))))

    H = 0
    c_ops = []
    for i in range(len(a)):
        H = H + w*a[i].dag() * a[i]
        c_ops.append(np.sqrt(kappa) * a[i])
    
    for j in range(len(s)):
        H = H + we_list[j] * s[j].dag() * s[j]
        c_ops.append(np.sqrt(gamma) * s[j])
        
    
    H = H + g * (s[0].dag()*a[0] + a[0].dag() * s[0])
    H = H + g * (s[1].dag()*a[0] + a[0].dag() * s[1])
    H = H + g * (s[2].dag()*a[1] + a[1].dag() * s[2])
    H = H + g * (s[3].dag()*a[1] + a[1].dag() * s[3])
    
    H = H - J * (a[0].dag() * a[1] + a[1].dag() * a[0])
    
    c_ops.append(np.sqrt(P) * a[0].dag())
    return H, w_list, c_ops, a, s, we_list

In [3]:
def run_spec(C, numem, spread, g, J, w, points):
    plt.close('all')
    Jems = False
    if C == 2:
#         H, w_list, c_ops, a, s = makeH_2_4(C, numem, spread, g, J, w, points)
        H_spread, w_list, c_ops_spread, a_spread,s_spread, we_list = makeH_2_4(C, numem, spread, g, J, w, points)
        print('Hamiltonians made')

        
    if C ==1: 
        H_spread, w_list, c_ops_spread, a_spread, s_spread = makeH_1_4(C, numem, spread, g, w, points)

#         H, w_list, c_ops, a, s = makeH_1_4(C, numem, spread, g, w, points)
        print('Hamiltonians made')

#     spec = spectrum(H, w_list, c_ops, a[0].dag(), a[0])
#     spec = spec - min(spec)
#     spec = spec/max(spec)
#     print('No spread finished')

    spec_spread = spectrum(H_spread, w_list, c_ops_spread, a_spread[0].dag(), a_spread[0])
    spec_spread =spec_spread - min(spec_spread)
    spec_spread = spec_spread/max(spec_spread)
#     plt.plot(w_list, spec_spread)
    return spec_spread, w_list
#     np.savetxt('checks/number cavs' + str(C) + '_emsinside' + str(4) + '_spread' +str(spread) + '_J' +str(J)+ '_g' +str(g) + '.txt', np.transpose([w_list, spec_spread]))

def run_eigens(C, numem, spread, g, J, w, points):
    if C == 2:
#         H, w_list, c_ops, a, s = makeH_2_4(C, numem, spread, g, J, w, points)
        H, w_list, c_ops, a, s, we_list = makeH_2_4(C, numem, spread, g, J, w, points)       
    if C ==1: 
        H_spread, w_list, c_ops_spread, a_spread, s_spread, we_list = makeH_1_4(C, numem, spread, g, w, points)

    eigens = H.eigenstates()
    return eigens[0], eigens[1], H, w_list, c_ops, a, s, we_list

def run_both(C, numem, spread, g, J, w, points):
    plt.close('all')

    H, w_list, c_ops, a,s, we_list = makeH_2_4(C, numem, spread, g, J, w, points)
#     print('Hamiltonians made')
    eigens = H.eigenstates()
    spec = spectrum(H, w_list, c_ops, a[0].dag(), a[0])
    spec =spec - min(spec)
    spec = spec/max(spec)
        

    return spec, w_list, eigens[0], eigens[1]

In [98]:
C = 2
numem = 4
spread = 0
g = 5
J = 0
counts = 1
w = 0
points = 100

In [7]:
eigenvals, eigenvects, H, w_list, c_ops, a, s, we_list = run_eigens(C, numem, 0, g, J, w, 100)

singlevals = []
singlevects = []
singleindex = []
singles = []
# print(eigens)
for ind in range(len(eigenvects)):
    npsi = a[0].dag()*a[0]*eigenvects[ind] + a[1].dag()*a[1] * eigenvects[ind] + s[0].dag()*s[0]*eigenvects[ind] + s[1].dag()*s[1]*eigenvects[ind] + s[2].dag()*s[2]*eigenvects[ind] + s[3].dag()*s[3]*eigenvects[ind]
#     print(ind)
#     print(npsi)
    if npsi == eigenvects[ind]:
        singlevals.append(np.real(eigenvals[ind]))
        singleindex.append(ind)
        state = []
        for j in range(64):
            state.append(eigenvects[ind][j][0][0])
        singles.append(np.real(np.transpose([eigenvals[ind], *state])))
#         print(state)
#         print(singles)

# np.savetxt('checks/diffs_J='+ str(J) + '_spread=0_all.txt', np.transpose([eigens[0]]), header = 'Cavities = ' + str(C) + 'Total Emitters = ' + str(numem) + 'g = ' + str(g), fmt = '%.8g')

# np.savetxt('checks/diffs_J='+ str(J) + '_spread=0_singlestates.txt', np.transpose(singles), header = 'Cavities = ' + str(C) + 'Total Emitters = ' + str(numem) + 'g = ' + str(g), fmt = '%.8g')


# for i in range(counts):
#     eigens, H, w_list, c_ops, a, s, we_list = run_eigens(C, numem, spread, g, J, w, 100)
    
#     singlevals = []
#     singlevects = []
#     singleindex = []
#     singles = []
#     for ind in range(len(eigenvals)):
#         npsi = a[0].dag()*a[0]*eigens[1][ind] + a[1].dag()*a[1] * eigens[1][ind] + s[0].dag()*s[0]*eigens[1][ind] + s[1].dag()*s[1]*eigens[1][ind] + s[2].dag()*s[2]*eigens[1][ind] + s[3].dag()*s[3]*eigens[1][ind]

#         if npsi == eigens[1][ind]:
#             singlevals.append(np.real(eigens[0][ind]))
#             singleindex.append(ind)
#             state = []
#             for j in range(64):
#                 state.append(eigens[1][ind][j][0][0])
#             singles.append(np.real(np.transpose([eigens[0][ind], *state])))
# #     if len(eigenvals) == 5:
# #         run_spec(C, numem, spread, g, J, w, points)
    
            
#     np.savetxt('checks/diffs_J='+ str(J) + '_spread=' + str(spread) + '_all_' + str(i) + '.txt', np.transpose([eigens[0]]), header = 'Cavities = ' + str(C) + 'Total Emitters = ' + str(numem) + 'g = ' + str(g) + 'spread = ' + str(we_list), fmt = '%.8g')
    
#     np.savetxt('checks/diffs_J='+ str(J) + '_spread=' + str(spread) + '_singlestates_' + str(i)+'.txt', np.transpose(singles), header = 'Cavities = ' + str(C) + 'Total Emitters = ' + str(numem) + 'g = ' + str(g) + 'spread = ' + str(we_list), fmt = '%.8g')
#     if i % 10 ==0:
#         print(i)

In [ ]:
psi0 = tensor(tensor(fock(2,1), fock(2,0), fock(2,0)), tensor(fock(2,0), fock(2,0), fock(2,0)))
for i in range(64):
    print(i, psi0[i])

In [ ]:
# psi0 = tensor(tensor(fock(2,0), fock(2,0), fock(2,0)), fock(2,0), fock(2,1), fock(2,1))
for i in range(64):
    print(i, singles[0][i])

In [ ]:
a[0]

In [ ]:
J = 10, no spread, spread = 5
empty
emitter 2
emitter 2 + emitter 3
cavity 1 + emitter 3
emitter 1 + emitter 3
emitter 0 + emitter 3
cavity 0 + emitter 3

J = 10, spread

In [6]:
C = 2
numem = 4
g = 10
# cavcoup = [1, 10, 100]
cavcoup = [10, 100]
counts = 30
w = 0
points = 1000
emspread = [g/4 * np.sqrt(C), g, 2*g]
# emspread = [g, 2*g]


for J in cavcoup:
    if J == 1: fname = 'spreadspecs/1J/'
    if J == 10: fname = 'spreadspecs/10J/'
    if J == 100: fname = 'spreadspecs/100J/'
    print(J)
    for spread in emspread:
        print(spread)
        for i in range(counts):
            print(i)
            spec, w_list, eigenvals, eigenvects = run_both(C, numem, spread, g, J, w, points)
            np.savetxt(fname + 'spec_spread' + str(spread) + '_' + str(i+20)+'.txt',  [spec, w_list])
            np.savetxt(fname + 'eigens_spread' + str(spread) + '_' + str(i+20)+'.txt',  np.transpose(eigenvals))


            fig, cx = plt.subplots(1,1)
            cx.plot(w_list, spec, linewidth=2, label = '$\delta$ = {}GHz')
#             cx.tick_params(axis='both', which='major', labelsize=24)  

#             cx.set_xlabel('$\omega/2\pi$ (GHz)',fontsize = 32)
#             cx.set_ylabel('Transmission (arb. u.)',fontsize = 32)
            fig.savefig(fname + 'spread' + str(spread) + '_' + str(i+20) + '.png')


10
3.5355339059327378
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
20
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
100
3.5355339059327378
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


OSError: exception: access violation reading 0x0000000000000008

In [ ]:
eigenvects

In [1]:
C = 2
numem = 4
g = 10
cavcoup = [1]
# cavcoup = [100]
counts = 30
w = 0
points = 1000
# emspread = [g/4 * np.sqrt(C), g, 2*g]
emspread = [2*g]


for J in cavcoup:
    if J == 1: fname = 'spreadspecs/1J/'
    if J == 10: fname = 'spreadspecs/10J/'
    if J == 100: fname = 'spreadspecs/100J/'
    print(J)
    for spread in emspread:
        print(spread)
        for i in range(counts):
            print(i)
            spec, w_list, eigenvals, eigenvects = run_both(C, numem, spread, g, J, w, points)
            np.savetxt(fname + 'spec_spread' + str(spread) + '_' + str(i+20)+'.txt',  [spec, w_list])
            np.savetxt(fname + 'eigens_spread' + str(spread) + '_' + str(i+20)+'.txt',  np.transpose(eigenvals))


            fig, cx = plt.subplots(1,1)
            cx.plot(w_list, spec, linewidth=2, label = '$\delta$ = {}GHz')
#             cx.tick_params(axis='both', which='major', labelsize=24)  

#             cx.set_xlabel('$\omega/2\pi$ (GHz)',fontsize = 32)
#             cx.set_ylabel('Transmission (arb. u.)',fontsize = 32)
            fig.savefig(fname + 'spread' + str(spread) + '_' + str(i) + '.png')

1
20
0


NameError: name 'run_both' is not defined